In [ ]:
import os
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df='/content/gdrive/MyDrive/Bone_Fracture.zip'

In [ ]:
df1=zipfile.ZipFile(df,'r')

In [ ]:
df1.extractall('/content/gdrive/MyDrive/Bone_Fracture')

In [ ]:
df1.close()

In [ ]:
os.listdir('/content/gdrive/MyDrive/Bone_Fracture')

['Negative', 'Positive']

In [ ]:
train_positive_dir = os.path.join('/content/gdrive/MyDrive/Bone_Fracture/Positive')


In [ ]:
train_negative_dir = os.path.join('/content/gdrive/MyDrive/Bone_Fracture/Negative')

In [ ]:
train_positive_names = os.listdir(train_positive_dir)
print(train_positive_names[2:5])


['003.jpg', '004.jpg', '005.jpg']


In [ ]:
train_negative_names = os.listdir(train_negative_dir)
print(train_negative_names[10:])


['094.jpg', '097.jpg', '110.jpg', '116.jpg', '138.jpg', '158.jpg', '159.jpg', '160.jpg', '165.jpg', '168.jpg', '171.jpg', '180.jpg', '181.jpg', '182.jpg', '208.jpg', '213.jpg', '225.jpg', '228.jpg', '229.jpg', '233.jpg', '240.jpg', '241.jpg', '245.jpg', '246.jpg', '247.jpg', '269.jpg', '270.jpg', '273.jpg', '277.jpg', '278.jpg', '283.jpg', '284.jpg', '285.jpg', '286.jpg', '289.jpg', '293.jpg', '294.jpg', '300.jpg', '301.jpg', '304.jpg', '305.jpg', '306.jpg', '312.jpg', '317.jpg', '322.jpg', '327.jpg', '330.jpg', '336.jpg', '337.jpg', '340.jpg', '345.jpg', '348.jpg', '349.jpg', '353.jpg', '354.jpg', '359.jpg', '362.jpg', '366.jpg', '377.jpg', '381.jpg', '382.jpg', '383.jpg', '385.jpg', '386.jpg', '389.jpg', '390.jpg', '395.jpg', '398.jpg', '402.jpg', '407.jpg', '419.jpg', '424.jpg', '426.jpg', '429.jpg', '432.jpg', '433.jpg', '439.jpg', '440.jpg', '441.jpg', '442.jpg', '457.jpg', '458.jpg', '465.jpg', '468.jpg', '474.jpg', '480.jpg', '486.jpg', '490.jpg', '496.jpg', '505.jpg', '506.jpg'

In [ ]:
print('total training normal images:', len(os.listdir(train_normal_dir)))
print('total training glaucoma images:', len(os.listdir(train_glaucoma_dir)))

total training normal images: 386
total training glaucoma images: 134


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

In [ ]:
pic_index = 0

In [ ]:
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

<Figure size 1152x1152 with 0 Axes>

In [ ]:

pic_index += 8
next_positive_pix = [os.path.join(train_positive_dir, fname) 
                for fname in train_positive_names[pic_index-8:pic_index]]
next_negative_pix = [os.path.join(train_negative_dir, fname) 
                for fname in train_negative_names[pic_index-8:pic_index]]

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)


In [ ]:
train_generator = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Bone_Fracture',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=8,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 169 images belonging to 2 classes.


In [ ]:
his=model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=100,
      verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/100
8/8 [==============================] - 27s 3s/step - loss: 0.9921 - acc: 0.4912
Epoch 2/100
8/8 [==============================] - 14s 2s/step - loss: 0.6914 - acc: 0.5156
Epoch 3/100
8/8 [==============================] - 8s 911ms/step - loss: 0.6928 - acc: 0.5312
Epoch 4/100
8/8 [==============================] - 4s 446ms/step - loss: 0.6796 - acc: 0.6667
Epoch 5/100
8/8 [==============================] - 6s 775ms/step - loss: 0.6431 - acc: 0.5938
Epoch 6/100
8/8 [==============================] - 5s 571ms/step - loss: 0.6831 - acc: 0.5781
Epoch 7/100
8/8 [==============================] - 4s 470ms/step - loss: 0.6578 - acc: 0.6094
Epoch 8/100
8/8 [==============================] - 3s 415ms/step - loss: 0.6150 - acc: 0.6316
Epoch 9/100
8/8 [==============================] - 4s 478ms/step - loss: 0.6140 - acc: 0.6406
Epoch 10/100
8/8 [==============================] - 4s 472ms/step - loss: 0.5538 - acc: 0.7500
Epoch 11/100
8/8 [==============================] - 4s 470ms/st

In [ ]:
model.save('/content/gdrive/MyDrive/bone_model.h5')